In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import imageio.v2 as imageio
import pickle
from tqdm import tqdm
from pathlib import Path
from scipy.interpolate import interp1d

from IPython.display import clear_output

from src.data.file_utils import GetTV

In [3]:
def nearest_index(array, value):
    """Find the index of the nearest value in an array."""
    return (np.abs(array - value)).argmin()

def crop_time(times, data, start_time, end_time):
    """Crop the data to the specified time range."""
    if data.ndim == 1:
        data = data.reshape(1, -1)
    start_idx = nearest_index(times, start_time)
    end_idx = nearest_index(times, end_time)
    return times[start_idx:end_idx], data[:,start_idx:end_idx]

def get_index(arr, coord):
    # Get the insertion indices
    ind = np.searchsorted(coord, arr)

    # Correct the indices to point to the nearest actual index
    ind = np.clip(ind, 0, len(coord) - 1)

    # Now, adjust the indices to get the closest value
    for i, cval in enumerate(arr):
        if ind[i] > 0 and abs(cval - coord[ind[i] - 1]) < abs(cval - coord[ind[i]]):
            ind[i] -= 1
            
    return ind

In [23]:
stem_path = 'all'

analysis_input_path = Path('../data/raw/tv_images') / stem_path
analysis_output_path = Path('../outputs/video/weighted_emission') / stem_path
analysis_output_path.mkdir(parents=True, exist_ok=True)
analysis_label_path = Path('../data/labels/weighted_emission') / stem_path
analysis_label_path.mkdir(parents=True, exist_ok=True)

tv = GetTV(analysis_input_path)
files = tv.list_files()
for idx, file in enumerate(files):
    print(idx, '\t',file.stem.split('_')[-1])

0 	 189057
1 	 189061
2 	 189062
3 	 189081
4 	 189088
5 	 189090
6 	 189093
7 	 189094
8 	 189097
9 	 189100
10 	 189101
11 	 189448
12 	 189451
13 	 190109
14 	 190110
15 	 190113
16 	 190114
17 	 190115
18 	 190116
19 	 199166
20 	 199171
21 	 199172
22 	 199351
23 	 199352
24 	 199353
25 	 199354


In [24]:
scaling_factor = 10
animated = False
save_label = True

In [28]:
for file_idx in range(len(files)):
    
    if animated:
        clear_output(wait=True)
    
    print(f"Processing Shot {files[file_idx].stem.split('_')[-1]}")
    gif_save_name = analysis_output_path / f'{files[file_idx].stem.split("_")[-1]}_animation.gif'
    mp4_save_name = analysis_output_path / f'{files[file_idx].stem.split("_")[-1]}_animation.mp4'
    
    [inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = tv.load_all(files[file_idx])
    inverted_times = vid_times[frames.astype(int)]
    inverted_dim = inverted.shape
    if (inverted_dim[1] != 201) or (inverted_dim[2] != 201):
        print('Resizing...')
        inverted = inverted[:,:201,:201]
        radii = radii[:,:201]
        elevation = elevation[:,:201]
    
    radii_hires = np.linspace(radii[0][0],radii[0][-1],len(radii[0])*scaling_factor)
    elevation_hires = np.linspace(elevation[0][0],elevation[0][-1],len(elevation[0])*scaling_factor)

    pkl_path = '../data/external/toksearch/detach.pkl'
    with open(pkl_path, 'rb') as file:
        points = pickle.load(file)
    point_keys = list(points.keys())
    shot = points[point_keys[file_idx]]['vars']
    times = points[point_keys[file_idx]]['time']
    times_transp = np.transpose(times)
    shot_transp = np.transpose(shot)
    
    crop_times, crop_shot = crop_time(times_transp, shot_transp, inverted_times[0], inverted_times[-1])
    rx = crop_shot[0] / 1e2
    zx = crop_shot[1] / 1e2
    rs = crop_shot[2] / 1e2
    zs = crop_shot[3] / 1e2

    interp_kind = 'linear'
    interpolator = interp1d(crop_times, rx, kind=interp_kind,
                            fill_value='extrapolate')
    rx_interp = interpolator(inverted_times)
    interpolator = interp1d(crop_times, zx, kind=interp_kind,
                            fill_value='extrapolate')
    zx_interp = interpolator(inverted_times)
    interpolator = interp1d(crop_times, rs, kind=interp_kind,
                            fill_value='extrapolate')
    rs_interp = interpolator(inverted_times)
    interpolator = interp1d(crop_times, zs, kind=interp_kind,
                            fill_value='extrapolate')
    zs_interp = interpolator(inverted_times)

    rx_idx = get_index(rx_interp, radii_hires) / scaling_factor
    zx_idx = get_index(zx_interp, elevation_hires) / scaling_factor
    zs_idx = get_index(zs_interp, elevation_hires) / scaling_factor

    z_arr_avg = []
    z_arr_ssa = []
    
    print("Calculating positions...")
    for idx, inverted_img in enumerate(inverted):
        filter_img = (inverted_img > 0.1) * inverted_img
        r = np.round(rx_idx[idx]).astype(int)
        sum_outer = np.sum(filter_img[:,r:],axis=1)
        indicies = np.arange(0, filter_img.shape[0])
        
        weighted_sum = np.sum(indicies * sum_outer)
        avg = np.divide(weighted_sum, np.sum(sum_outer), out=np.empty_like(weighted_sum), where=np.sum(sum_outer)!=0)
        z_arr_avg.append(avg)
        
        weighted_square_sum = np.sum((indicies * sum_outer)**2)
        total_square_weight = np.sum(sum_outer**2)
        sqrt_sum_sq_avg = np.divide(weighted_square_sum, total_square_weight, out=np.empty_like(weighted_square_sum), where=total_square_weight!=0)**0.5
        z_arr_ssa.append(sqrt_sum_sq_avg)
    
    z_arr_avg = np.asarray(z_arr_avg)
    z_arr_ssa = np.asarray(z_arr_ssa)
    avg_avg_idx = np.mean(z_arr_avg)
    ssa_avg_idx = np.mean(z_arr_ssa)
    z_arr_avg[np.where(z_arr_avg == 0)[0]] = avg_avg_idx
    z_arr_ssa[np.where(z_arr_ssa == 0)[0]] = ssa_avg_idx

    z_avg_rounds = [z*scaling_factor for z in z_arr_avg]
    z_ssa_rounds = [z*scaling_factor for z in z_arr_ssa]

    z_avg_real = elevation_hires[np.round(z_avg_rounds).astype(int)]
    z_ssa_real = elevation_hires[np.round(z_ssa_rounds).astype(int)]
    
    print("Sample Values: ",z_avg_real[:6])
    
    if save_label:
        print("Saving PKLs...")
        with open(analysis_label_path / f'{files[file_idx].stem}.pkl', 'wb') as file:
            pickle.dump(z_ssa_real, file)
    
    if animated:
        # Initialize figure and axes
        print("Animating...")
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        
        # Initial plots
        line_rx, = ax[0].plot(rx_idx)
        vline_rx = ax[0].axvline(0, color='r')
        ax[0].set_ylim(0, inverted_dim[1])
        ax[0].set_title('$r_X$')

        line_avg, = ax[1].plot(z_arr_avg, label='avg')
        line_ssa, = ax[1].plot(z_arr_ssa, label='ssa')
        vline_z = ax[1].axvline(0, color='r')
        ax[1].legend(loc='upper right')
        ax[1].set_title('Z position')

        img = ax[2].imshow(filter_img, origin='lower')
        rect = patches.Rectangle((rx_idx[0], 0), inverted_dim[1]-rx_idx[0]-1, inverted_dim[2]-1, linewidth=1, edgecolor='w', facecolor='none')
        ax[2].add_patch(rect)
        vspan = ax[2].axvspan(0, rx_idx[0], color='black', alpha=0.7)
        hline_ssa = ax[2].axhline(z_arr_ssa[0], c='lime', label='ssa')
        hline_zx = ax[2].axhline(zx_idx[0], c='r', label='$z_X$')
        hline_zs = ax[2].axhline(zs_idx[0], c='r', label='$z_S$')
        ax[2].legend(loc='upper right')
        ax[2].set_title(f'Inverted View: 0')

        fig.suptitle(f"Shot {files[file_idx].stem.split('_')[-1]}")

        frames = []
        # Function to update the plot
        def update(idx):
            img.set_data(inverted[idx])
            vline_rx.set_xdata([idx])
            vline_z.set_xdata([idx])
            
            rect.set_x(rx_idx[idx])
            vspan.set_xy((0, 0))
            vspan.set_width(rx_idx[idx])
            
            hline_ssa.set_ydata([z_arr_ssa[idx]])
            hline_zx.set_ydata([zx_idx[idx]])
            hline_zs.set_ydata([zs_idx[idx]])
            
            ax[2].set_title(f'Inverted View: {idx}')
            
            return vline_rx, vline_z, rect, vspan, hline_ssa, hline_zx, hline_zs
            
        # Create the animation using FuncAnimation
        ani = animation.FuncAnimation(fig, update, frames=range(inverted_dim[0]), blit=True, repeat=False)

        # Save the animation as an MP4 file
        print("Saving MP4...")
        FFwriter = animation.FFMpegWriter(fps=30, extra_args=["-vcodec", "libx264"])
        ani.save(mp4_save_name, writer=FFwriter)

        plt.close(fig)

Processing Shot 189057
Calculating positions...
Sample Values:  [-1.1879542 -1.1879542 -1.1914386 -1.1889497 -1.1979094 -1.1859632]
Saving PKLs...
Processing Shot 189061
Calculating positions...
Sample Values:  [-1.1187656 -1.1187656 -1.1098058 -1.1003485 -1.1088103 -1.1058238]
Saving PKLs...
Processing Shot 189062
Calculating positions...
Sample Values:  [-1.1073171 -1.0993528 -1.0983574 -1.1013439 -1.1008462 -1.0993528]
Saving PKLs...
Processing Shot 189081
Calculating positions...
Sample Values:  [-1.1685416 -1.1590841 -1.1630661 -1.1680437 -1.1775012 -1.1735191]
Saving PKLs...
Processing Shot 189088
Calculating positions...
Sample Values:  [-1.1585864 -1.1605774 -1.1700349 -1.1536088 -1.15112   -1.1620706]
Saving PKLs...
Processing Shot 189090
Calculating positions...
Sample Values:  [-1.1695371 -1.1784967 -1.1670482 -1.1580886 -1.174017  -1.17999  ]
Saving PKLs...
Processing Shot 189093
Calculating positions...
Sample Values:  [-1.1914386 -1.1829766 -1.1889497 -1.1844698 -1.185465